<a href="https://colab.research.google.com/github/franklinwillemen/Sherlock-Hemlock-Graph-Based-NER/blob/main/data_build_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Create a symbolic link to the Colab Notebooks directory
nb_path = '/content/notebooks'

# Check if the symbolic link or directory already exists
if not os.path.exists(nb_path):
    os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)

# Add the symbolic link to the system path
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [18]:
import logging
import transformers as t
import datasets
from datasets import load_dataset
import torch
import torch_geometric
import numpy as np
import pandas as pd
import spacy

In [19]:
dataset = load_dataset("joelito/mapa")

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['language', 'type', 'file_name', 'sentence_number', 'tokens', 'coarse_grained', 'fine_grained'],
        num_rows: 27823
    })
    test: Dataset({
        features: ['language', 'type', 'file_name', 'sentence_number', 'tokens', 'coarse_grained', 'fine_grained'],
        num_rows: 10590
    })
    validation: Dataset({
        features: ['language', 'type', 'file_name', 'sentence_number', 'tokens', 'coarse_grained', 'fine_grained'],
        num_rows: 3354
    })
})

In [21]:
np.unique(dataset["train"]["language"])

array(['bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'ga',
       'hu', 'it', 'lt', 'lv', 'mt', 'nl', 'pt', 'ro', 'sk', 'sv'],
      dtype='<U2')

In [15]:
df = pd.DataFrame(dataset["train"].to_pandas())

In [16]:
df

,language,type,file_name,sentence_number,tokens,coarse_grained,fine_grained
0,bg,EUR-LEX,204645_bg,0,"[Преюдициално, запитване, от, Rechtbank, Amste...","[O, O, O, B-ORGANISATION, I-ORGANISATION, O, B...","[O, O, O, O, O, O, B-COUNTRY, O, O, O, O, B-DA..."
1,bg,EUR-LEX,204645_bg,1,"[(, Дело, C, -, 314, /, 18, )]","[O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O]"
2,bg,EUR-LEX,204645_bg,2,"[Език, на, производството, :, нидерландски]","[O, O, O, O, O]","[O, O, O, O, O]"
3,bg,EUR-LEX,204645_bg,3,"[Запитваща, юрисдикция]","[O, O]","[O, O]"
4,bg,EUR-LEX,204645_bg,4,"[Rechtbank, Amsterdam]","[B-ORGANISATION, I-ORGANISATION]","[O, O]"
...,...,...,...,...,...,...,...
27818,sv,EUR-LEX,205521_sv.txt,219,"[–, att, en, kyrka, eller, en, annan, organisa...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
27819,sv,EUR-LEX,205521_sv.txt,220,"[–, att, en, särbehandling, ,, när, det, gälle...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
27820,sv,EUR-LEX,205521_sv.txt,221,"[2, ), En, nationell, domstol, ,, som, har, at...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
27821,sv,EUR-LEX,205521_sv.txt,222,[Underskrifter],[O],[O]


In [25]:
# List of language codes to drop
languages_to_drop = ["ga", "mt", "sk"]

# Drop the rows with the specified language codes
mapa_df = df[~df['language'].isin(languages_to_drop)]

# Reset the index of the filtered DataFrame
mapa_df.reset_index(drop=True, inplace=True)

In [27]:
nlner = mapa_df[mapa_df["language"] == "nl"]

In [30]:
nlner

,language,type,file_name,sentence_number,tokens,coarse_grained,fine_grained
18698,nl,EUR-LEX,204645_nl,0,"[Verzoek, om, een, prejudiciële, beslissing, ,...","[O, O, O, O, O, O, O, O, O, O, B-ADDRESS, O, B...","[O, O, O, O, O, O, O, O, O, O, B-CITY, O, B-CO..."
18699,nl,EUR-LEX,204645_nl,1,"[(, Zaak, C, -, 314, /, 18, )]","[O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O]"
18700,nl,EUR-LEX,204645_nl,2,"[Procestaal, :, Nederlands]","[O, O, O]","[O, O, O]"
18701,nl,EUR-LEX,204645_nl,3,"[Verwijzende, rechter]","[O, O]","[O, O]"
18702,nl,EUR-LEX,204645_nl,4,"[Rechtbank, Amsterdam]","[O, B-ADDRESS]","[O, B-CITY]"
...,...,...,...,...,...,...,...
20084,nl,EUR-LEX,205521_nl,196,"[–, enerzijds, ,, een, kerk, of, een, andere, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
20085,nl,EUR-LEX,205521_nl,197,"[–, anderzijds, ,, een, verschillende, behande...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
20086,nl,EUR-LEX,205521_nl,198,"[2, ), Een, nationale, rechterlijke, instantie...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
20087,nl,EUR-LEX,205521_nl,199,[ondertekeningen],[O],[O]


In [32]:
!python -m spacy download nl_core_news_lg

2023-04-08 16:58:19.233341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')


In [34]:
nlp = spacy.load("nl_core_news_lg")

In [36]:
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f2aef5f23a0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f2aef5f2b20>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7f2aef5f2700>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f2aefd894a0>),
 ('lemmatizer',
  <spacy.pipeline.edit_tree_lemmatizer.EditTreeLemmatizer at 0x7f2aef5f22e0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f2aef5be740>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f2aefd89270>)]

In [55]:
texts = [
    "Mijn moeder was eergisteren jarig",
    "Wat moeten we nou met al die appels.",
]

for text in texts:
  doc = nlp(text)
  for token in doc:
    print(token.pos_)

PRON
NOUN
AUX
ADV
ADJ
PRON
AUX
PRON
ADV
ADP
ADV
DET
NOUN
PUNCT


In [ ]:
# Create a new DataFrame with the word tokens and their corresponding labels
tokens_and_labels = []
for index, row in enner.iterrows():
    tokens = row["input"]
    labels = row["label"]
    for token, label in zip(tokens, labels):
        tokens_and_labels.append({"token": token, "label": label})

tokens_labels_df = pd.DataFrame(tokens_and_labels)

print(tokens_labels_df.head())

         token  label
0      Request      0
1          for      0
2            a      0
3  preliminary      0
4       ruling      0


In [ ]:
tokens_labels_df[tokens_labels_df["label"] == 39]["token"].unique()

array(['Mr', 'Ms'], dtype=object)

In [ ]:
tokens_labels_df["label"].unique()

array([ 0,  5, 21, 19, 15, 27, 39, 35, 36, 29, 33, 34, 23, 24, 13, 30, 12,
        3,  7,  8, 28,  9,  6,  4, 10, 37, 25, 26])

In [ ]:
tokens_labels_df["token"].unique()

array(['Request', 'for', 'a', ..., 'derived', 'confer', 'rely'],
      dtype=object)